# Interactive maps¶
In this tutorial we will learn how to publish data from Python on interactive leaflet.js maps.

JavaScript (JS) is a programming language for adding interactive content (such a zoomamble maps!) on webpages. Leaflet is a popular JavaScript library for creating interactive maps for webpages (OpenLayers is another JavaScript library for the same purpose).

Here, will mainly focus on Folium - a Python library that makes it easy to convert data from (Geo)DataFrames into interactive Leaflet maps.

Other interesting libraries for creating interactive visualizations from spatial data:

- mapboxgl

- Bokeh

- Geoviews

- plotly express

# Creating a simple interactive web-map¶
- Import folium and other useful packages:

In [27]:
import folium

In [28]:
from pyproj import CRS
import geopandas as gpd
import matplotlib.pyplot as plt

We will start by creating a simple interactive web-map without any data on it. We just visualize OpenStreetMap on a specific location of the world.

- First thing that we need to do is to create a Map instance and define a location for zooming in the data:

In [29]:
#create a map instance
m = folium.Map(location= [ 60.25, 24.8 ] , zoom_start=10, control_scale=True)

- The first parameter location takes a pair of lat, lon values as list as an input which will determine where the map will be positioned when user opens up the map. 
- zoom_start -parameter adjusts the default zoom-level for the map (the higher the number the closer the zoom is). 
- control_scale defines if map should have a scalebar or not.

Let’s see what our map looks like:

In [30]:
m

In [31]:
outfp = "base_map.html"
m.save(outfp)

- You should now see a html file in your working directory. 
- You can open the file in a web browser in order to see the map or in a text editor in order to see the source definition

Let’s create another map with different settings (location, bacground map, zoom levels etc). 
See documentation of the Map() object for all avaiable options.

- tiles -parameter is used for changing the background map provider and map style (see the documentation for all in-built options).

In [32]:
#change the base map to 'Stamen Toner'
m = folium.Map(location=[40.730610, -73.935242], tiles = 'Stamen Toner',
                zoom_start=12, control_scale=True, prefer_canvas=True )

m

# Adding layers to the map

- Add a simple marker on the webmap

In [33]:
#Create a map instance
m= folium.Map(location= [60.20, 24.96],
                  zoom_start = 12, 
                      control_scale=True)

#add marker 
#run help(folium.Icon) for more info about icons

#create a marker at 'location lat long, 
#text_box = Kumpula Campus, 
#symobology= green marker with ok_sign inside
#add to the map
folium.Marker(
                location= [60.20426,24.96179],
                    popup="Kumpula Campus",
                        icon=folium.Icon(color='green', icon='ok-sign') ).add_to(m)

#show on map
m

 Folium combines the strenghts of data manipulation in Python with the mapping capabilities of Leaflet.js.

- plot data from a geodataframe using folium.

In [34]:
#file path
points_fp = r"addresses.shp"

#read the data
points= gpd.read_file(points_fp)

#check the input data
points.head()

,address,id,geometry
0,"Kampinkuja 1, 00100 Helsinki, Finland",1001,POINT (24.93017 60.16837)
1,"Kaivokatu 8, 00101 Helsinki, Finland",1002,POINT (24.94189 60.16987)
2,"Hermanstads strandsväg 1, 00580 Helsingfors, F...",1003,POINT (24.97740 60.18736)
3,"Itäväylä, 00900 Helsinki, Finland",1004,POINT (25.09196 60.21448)
4,"Tyynenmerenkatu 9, 00220 Helsinki, Finland",1005,POINT (24.92148 60.15658)


- convert points from geodataframe to geojson

In [35]:
points_gjson = folium.features.GeoJson(points, name="Public transport stations")

In [36]:
#check GeoJSON features
points_gjson.data.get('features')

[{'id': '0',
  'type': 'Feature',
  'properties': {'address': 'Kampinkuja 1, 00100 Helsinki, Finland',
   'id': 1001},
  'geometry': {'type': 'Point', 'coordinates': [24.9301701, 60.1683731]},
  'bbox': [24.9301701, 60.1683731, 24.9301701, 60.1683731]},
 {'id': '1',
  'type': 'Feature',
  'properties': {'address': 'Kaivokatu 8, 00101 Helsinki, Finland',
   'id': 1002},
  'geometry': {'type': 'Point', 'coordinates': [24.9418933, 60.1698665]},
  'bbox': [24.9418933, 60.1698665, 24.9418933, 60.1698665]},
 {'id': '2',
  'type': 'Feature',
  'properties': {'address': 'Hermanstads strandsväg 1, 00580 Helsingfors, Finland',
   'id': 1003},
  'geometry': {'type': 'Point',
   'coordinates': [24.9774004, 60.18735880000001]},
  'bbox': [24.9774004, 60.18735880000001, 24.9774004, 60.18735880000001]},
 {'id': '3',
  'type': 'Feature',
  'properties': {'address': 'Itäväylä, 00900 Helsinki, Finland', 'id': 1004},
  'geometry': {'type': 'Point',
   'coordinates': [25.0919641, 60.21448089999999]},
  'b

- Now we have our population data stored as GeoJSON format which basically contains the data as text in a similar way 
  that it would be written in the .geojson file.
  
Add the points onto the Helsinki basemap:

In [37]:
#create a map instance
#start_location is 60.25,24.8
#basemap= cartodbpositron
#zoom level is 11
m = folium.Map(location=[60.25, 24.8], tiles='cartodbpositron', zoom_start=11, control_scale = True)

#add points to the map instance
#points_gjson.add_to(m)

#alternative syntax for adding points to map instance
m.add_child(points_gjson)

# Layer Control

- LayerControl() object allows the user to control which map layers are visible

In [38]:
#Create a layer control object and add it to our map instance
folium.LayerControl().add_to(m)

#Show map
m

# Heat Map

Folium plugins allow for use of popular tools available in leaflet. One of the plugins is a heatmap, which creates a heatmap layer from input points.

Lets visualize a heatmap of the public transport stations in Helsinki using the addresses input data.
folium.plugins.Heatmap requires a list of points, or numpy arry as input, so we first need to manipulate the data.

In [39]:
#get x and y coordinates for each point and create a new corresponding coordinate column to store the coordinate
points['x'] = points['geometry'].x
points['y'] = points['geometry'].y

#create a list of coordinate pairs
locations = list( zip ( points['y'], points['x'] ) )

In [40]:
#check the data
locations

[(60.1683731, 24.9301701),
 (60.1698665, 24.9418933),
 (60.18735880000001, 24.9774004),
 (60.21448089999999, 25.0919641),
 (60.1565781, 24.9214846),
 (60.23489060000001, 25.0816923),
 (60.2033879, 25.042239),
 (60.2753891, 25.035855),
 (60.2633799, 25.0291078),
 (60.22243630000001, 24.8718598),
 (60.1711874, 24.94251),
 (60.2306474, 24.8840504),
 (60.240163, 24.877383),
 (60.22163339999999, 24.9483202),
 (60.25149829999999, 25.0125655),
 (60.2177823, 24.893153),
 (60.2485471, 24.86186),
 (60.2291135, 24.9670533),
 (60.1986856, 24.9334051),
 (60.22401389999999, 24.8609335),
 (60.2436961, 24.9934979),
 (60.24444239999999, 25.040583),
 (60.20966609999999, 25.0778094),
 (60.20751019999999, 25.1424936),
 (60.225599, 25.0756547),
 (60.2382054, 25.1080054),
 (60.18789030000001, 24.9609122),
 (60.19413939999999, 25.0291263),
 (60.18837519999999, 25.0068399),
 (60.1793862, 24.9494874),
 (60.1694809, 24.9337569),
 (60.16500139999999, 24.9250072),
 (60.159069, 24.9214046),
 (60.1719108, 24.946851

In [41]:
from folium.plugins import HeatMap

#create a map instance
m = folium.Map(locations=[60.25,24.8], tiles ='stamentoner', zoom_start=10, control_scale=True)

#Add heatmap to map instance
#available parameters:  HeatMap(data, name=None, min_opacity=0.5, max_zoom=18, max_val=1.0, radius=25, blur=15, gradient=None, overlay=True, control=True, show=True)
HeatMap(locations).add_to(m)

#show map
m

# Clustered point map

Lets visualize the address points (locations of transport stations in Helsinki) on top of the choropleth map using clustered markers
using fliums MarkerCluster class.

In [42]:
from folium.plugins import MarkerCluster

In [43]:
#Create a map instance
m = folium.Map(location=[60.25,24.8], tiles='cartodbpositron', zoom_start=11, control_scale=True)

In [44]:
#Following this example:  https://github.com/python-visualization/folium/blob/master/examples/MarkerCluster.ipynb

#get x and y coordinates for each point
points['x'] = points['geometry'].x
points['y'] = points['geometry'].y

#create a list of coordinate pairs
locations= list( zip (points['y'], points['x'] ) )

In [45]:
#Create a folium marker cluster
marker_cluster = MarkerCluster(locations)

#add marker cluster to map
marker_cluster.add_to(m)

#show map
m

# Choropleth Map
Next, let's check how we can overlay a population map on top of a basemap using folium's choropleth method.
- This method is able to read the geometries and attributes directly from a geodataframe.

This map is an modified example taken from folium quickstart: https://python-visualization.github.io/folium/quickstart.html#Choropleth-maps

- read in the population grid from HSY wfs

In [46]:
import geopandas as gpd
from pyproj import CRS
import requests
import geojson

#specify the url for the web feature service
url = 'https://kartta.hsy.fi/geoserver/wfs'

#specify the parameters (read data in json format)
params = dict(service='WFS', 
              version='2.0.0',
              request= 'GetFeature',
              typeName='asuminen_ja_maankaytto:Vaestotietoruudukko_2020',
              outputFormat='json')

#Fetch data from WFS using requests
r = requests.get(url, params=params )

#create geodataframe from geojson
data = gpd.GeoDataFrame.from_features(geojson.loads(r.content))

#clean overall cell
data = data[data['index'] != 27699]

#check the data
data.head()

,geometry,index,asukkaita,asvaljyys,ika0_9,ika10_19,ika20_29,ika30_39,ika40_49,ika50_59,ika60_69,ika70_79,ika_yli80
0,"POLYGON ((25472499.995 6685998.998, 25472499.9...",703,5,51,99,99,99,99,99,99,99,99,99
1,"POLYGON ((25472499.995 6684249.004, 25472499.9...",710,8,44,99,99,99,99,99,99,99,99,99
2,"POLYGON ((25472499.995 6683999.005, 25472499.9...",711,5,90,99,99,99,99,99,99,99,99,99
3,"POLYGON ((25472499.995 6682998.998, 25472499.9...",715,13,34,99,99,99,99,99,99,99,99,99
4,"POLYGON ((25472749.993 6690249.003, 25472749.9...",848,5,53,99,99,99,99,99,99,99,99,99


In [47]:
from pyproj import CRS

#define crs
data.crs = CRS.from_epsg(3879)

- Reproject to WGS84 (epsg:4326 )

In [48]:
#reproject to WGS84
data = data.to_crs(epsg=4326)

#check layer crs definition
print(data.crs)

epsg:4326


Rename columns

In [49]:
#change column name
data = data.rename(columns={'asukkaita': 'pop20'} )

#create a geo-id which is needed by folium instance (it needs to have a unique identifier for each row)
data['geoid'] = data.index.astype(str)

#select only needed columns
data = data[ ['geoid', 'pop20', 'geometry'] ]

#convert to geojson (not needed for simple choropleth map)
#pop_json= data.to_json()

#check the data
data.head()

,geoid,pop20,geometry
0,0,5,"POLYGON ((24.50287 60.28562, 24.50284 60.28787..."
1,1,8,"POLYGON ((24.50311 60.26992, 24.50308 60.27216..."
2,2,5,"POLYGON ((24.50315 60.26767, 24.50311 60.26992..."
3,3,13,"POLYGON ((24.50328 60.25870, 24.50325 60.26094..."
4,4,5,"POLYGON ((24.50682 60.32378, 24.50678 60.32603..."


Create an interactive choropleth map from the population grid

In [55]:
#create a map instance
m = folium.Map(location=[60.25, 24.8], tiles = 'cartodbpositron', zoom_start=10, control_scale=True)

#create a choropleth plot add to map instance
#notice: 'geoid' column that we created earlier needs to be assigned always as the first column
folium.Choropleth(
                  geo_data=data,
                  name='Population in 2020',
                  data=data,
                  columns= ['geoid','pop20'],
                  key_on='feature.id',
                  fill_color='YlOrRd',
                  fill_opacity=0.7,
                  line_opacity=0.2,
                  line_color='white',
                  line_weight=0,
                  highlight=False,
                  smooth_factor=1.0,
                  #threshold_scale =[100,250,500,1000,2000],
                  legend_name= 'Population in Helsinki').add_to(m)

#show map
m

# Tool Tips